In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import mediapipe as mp
import os
import pandas as pd
import numpy as np



/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


In [ ]:
#Initialize Mediapipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
# Set up the Hand model
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Define the Landmark Extraction Function
def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not read image {image_path}")
        return None

    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image
    results = hands.process(image_rgb)

    # Check if a hand was detected
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        # Create a list to store the (x, y, z) coordinates
        landmark_list = []
        for landmark in hand_landmarks.landmark:
            landmark_list.append(landmark.x)
            landmark_list.append(landmark.y)
            landmark_list.append(landmark.z)
        return landmark_list
    else:
        return None

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/Indian sign language"
classes = ['A', 'B', 'C', 'L', 'V', 'W', '3', '5', '7', '9']

In [ ]:
landmarks_all = []
labels_all = []

print("Starting landmark extraction...")

for label in classes:
    class_path = os.path.join(DATASET_PATH, label)

    if not os.path.isdir(class_path):
        print(f"Warning: '{label}' is not a valid directory. Skipping.")
        continue

    print(f"Processing class: {label}")

    # Loop over every image in the class folder
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)

        # Extract the landmarks
        landmarks = extract_landmarks(image_path)

        # If landmarks were successfully extracted, add them
        if landmarks:
            landmarks_all.append(landmarks)
            labels_all.append(label)

print("...Landmark extraction complete.")



Starting landmark extraction...
Processing class: A
Processing class: B
Processing class: C
Processing class: L
Processing class: V
Processing class: W
Processing class: 3
Processing class: 5
Processing class: 7
Processing class: 9
...Landmark extraction complete.


In [ ]:
#Save the New Dataset to a CSV File
landmark_names = []
for i in range(21):
    landmark_names.append(f'x{i}')
    landmark_names.append(f'y{i}')

    landmark_names.append(f'z{i}')

# Create the DataFrame from our collected data
df = pd.DataFrame(landmarks_all, columns=landmark_names)

# Insert the 'label' column at the beginning
df.insert(0, 'label',labels_all)

# Save to CSV
filename = 'hand_landmarks.csv'
df.to_csv(filename, index=False)

print(f"Successfully created landmark dataset: {filename}")
print(f"Total samples processed: {len(df)}")
print(df.head())

hands.close()

Successfully created landmark dataset: hand_landmarks.csv
Total samples processed: 12200
  label        x0        y0            z0        x1        y1        z1  \
0     A  0.823453  0.579103 -3.187604e-07  0.737825  0.529141 -0.029101   
1     A  0.169535  0.589117 -3.636876e-07  0.244986  0.519247 -0.040643   
2     A  0.813743  0.599789 -4.762738e-07  0.716918  0.548508 -0.043591   
3     A  0.820679  0.532305 -3.481125e-07  0.733561  0.483653 -0.031798   
4     A  0.800146  0.520467 -3.952541e-07  0.715375  0.473747 -0.038439   

         x2        y2        z2  ...       z17       x18       y18       z18  \
0  0.644049  0.480069 -0.058952  ... -0.118808  0.732333  0.763040 -0.145423   
1  0.361213  0.454578 -0.069290  ... -0.131461  0.335835  0.736441 -0.147996   
2  0.632174  0.482998 -0.069572  ... -0.074553  0.815737  0.687996 -0.091948   
3  0.648944  0.424221 -0.062947  ... -0.098653  0.753291  0.664833 -0.113809   
4  0.635758  0.414098 -0.071736  ... -0.110810  0.749369  0.